In [3]:
import qutip as qt
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy.integrate as integrate

from tqdm.notebook import trange, tqdm
import pickle
import time
from itertools import product
# from skimage.transform import iradon


In [4]:
for i in trange(3, desc="Outer loop", bar_format="{l_bar}{bar} {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]"):
    for j in trange(3, desc=f"Inner loop {i}", leave=False):
        for k in trange(5, desc=f"Nested loop {i}.{j}", leave=False):
            # Simulate some work
            time.sleep(0.1)

Outer loop:   0%|           0/3 [00:00<?, ?it/s]

Inner loop 0:   0%|          | 0/3 [00:00<?, ?it/s]

Nested loop 0.0:   0%|          | 0/5 [00:00<?, ?it/s]

Nested loop 0.1:   0%|          | 0/5 [00:00<?, ?it/s]

Nested loop 0.2:   0%|          | 0/5 [00:00<?, ?it/s]

...

Nested loop 1.2:   0%|          | 0/5 [00:00<?, ?it/s]

Inner loop 2:   0%|          | 0/3 [00:00<?, ?it/s]

Nested loop 2.0:   0%|          | 0/5 [00:00<?, ?it/s]

Nested loop 2.1:   0%|          | 0/5 [00:00<?, ?it/s]

Nested loop 2.2:   0%|          | 0/5 [00:00<?, ?it/s]

In [5]:
# helpter functions to format strings
def format_complex(val):
    """    Format a complex number into a string representation.

    Args:
        val (complex): The complex number to format.

    Returns:
        outstr (str): A string representation of the complex number, formatted to two decimal places.
        If the real part is zero, it will only show the imaginary part. If both parts are zero, it returns "0". 
        The imaginary part is prefixed with '+' or '-' depending on its sign.
        The output is padded with spaces on both sides.
    """
    outstr = ""
    real_part = val.real
    imag_part = val.imag
    if real_part != 0:
        outstr += f"{real_part:.2f}"
    if imag_part != 0:
        if imag_part > 0:
            outstr += f"+{imag_part:.2f}j"
        else:
            outstr += f"-{abs(imag_part):.2f}j"
    return " " + outstr + " " if outstr else "0"

# Reconstruction

### TO DO:
- make reconstruction 
- compute fidelity
- compare different estimators (MLE, etc.)
- compare using different number of angles -- currently using 6 angles in $[0, \frac{5\pi}{6}]$
- compare using different binning for $X_\theta$

## Read data

In [6]:
data = np.load("measurements.npz")
thetas = data["thetas"]
measurements = data["measurements"]

with np.printoptions(precision=3, linewidth=150, edgeitems=10, suppress=True):
    print("      thetas:", thetas)
    print("measurements:", measurements)

# display metadata using bash command
!cat measurements_metadata.txt


      thetas: [0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    ... 2.618 2.618 2.618 2.618 2.618 2.618 2.618 2.618 2.618 2.618]
measurements: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ... 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


'cat' is not recognized as an internal or external command,
operable program or batch file.


## Background
The homodyne tomography experiment gives a set of data $(X_\theta, \theta)$  -- see [A. I. Lvovsky](../references/RevModPhys.81.299.pdf)

The data is generated in the [generateState notebook](generateState.ipynb) and save to the [data file](measurements.npz) and the [meta data file](measurements_metadata.txt)

### Radon transform

In [7]:
def inverse_radon(X, theta, xmax=10, pmax=10, res=101):
    
    x = np.linspace(-xmax, xmax, res)
    p = np.linspace(-pmax, pmax, res)
    X, P = np.meshgrid(x, p)

    N = len(X)
    norm = 1/(2*np.pi**2*N)
    
    pass

### Reconstruction using Maximum Likelihood Estimator 
Based on [A. I. Lvovsky - section III.B](../references/RevModPhys.81.299.pdf)

The probability of measuring $j$ from a density matrix $\hat{\rho}$ is $\mathrm{P}_\rho(j) = \mathrm{Tr}\left[\hat{\Pi}_j\hat{\rho}\right]$ where $\hat{\rho}$ is the maximum 

In [30]:
def quadrature_operator(N : int = 50, theta : float = 0.0) -> qt.Qobj:
    """Generate the quadrature operator for a given angle theta.

    Args:
        N (int): Dimension of the truncated Hilbert space.
        theta (float): Angle in radians for the quadrature operator.
    Returns:
        qt.Qobj: The quadrature operator corresponding to the angle theta.
    """
    a = qt.destroy(N)
    term = a * np.exp(-1j * theta)
    
    return (term + term.dag()) / np.sqrt(2)
        

def build_bins(N, thetas, n_bins=60, x_edges=None, tol=1e-12):
    """Build bins for the quadrature operator eigenstates based on the provided angles.

    Args:
        N (int): Dimension of the Hilbert space.
        thetas (ndarray): Array of angles in radians for which to compute the quadrature operators.
        n_bins (int, optional): Number of bins to create for the eigenvalues. Defaults to 60.
        x_edges (ndarray, optional): Edges for the bins. If `None`, they will be calculated based on the eigenvalues. 

    Returns:
        povms: List of POVMs (Positive Operator-Valued Measures) corresponding to the quadrature operators.
        x_edges: Edges of the bins.
        spectra: List of eigenvalues and eigenvectors for each quadrature operator. 
    """
    spectra = [quadrature_operator(N, theta).eigenstates() for theta in thetas]
    if x_edges is None: # If no edges are provided, calculate them based on the eigenvalues
        # Find the minimum and maximum eigenvalues across all spectra
        xmin = min(e.min() for e, _ in spectra)
        xmax = max(e.max() for e, _ in spectra) 
        pad = 1e-9 # Padding to avoid numerical issues
        x_edges = np.linspace(xmin - pad, xmax + pad, n_bins + 1) 
    
    povms = [] 
    for evals, evecs in tqdm(spectra, desc="Building bins", total=len(spectra)): 
        bin_indices = np.digitize(evals, x_edges) - 1  # -1 to convert to zero-based index
        E_bins = [qt.Qobj(np.zeros((N,N), dtype=np.complex128)) for _ in range(n_bins)] # Initialize bins with zero matrices
        # Fill the bins with the eigenvectors
        for i, bin in enumerate(bin_indices):
            if bin < 0: bin = 0 # Ensure bin index is within bounds
            if bin >= n_bins: bin = n_bins - 1 # Ensure bin index is within bounds
            E_bins[bin] += evecs[i] * evecs[i].dag() 
        povms.append(E_bins)
    return povms, x_edges, spectra

povms, edges, spectra = build_bins(N=50, thetas=thetas, n_bins=60)
len(povms), len(povms[0]), len(edges), len(spectra), len(spectra[0][0]), len(spectra[0][1])

Building bins:   0%|          | 0/1212 [00:00<?, ?it/s]

(1212, 60, 61, 1212, 50, 50)

In [17]:
def bin_samples(samples, thetas, x_edges, tol=1e-9):
    """This function takes samples and thetas, and bins them into quadrature bins defined by x_edges.

    Args:
        samples (ndarray): Array of samples to be binned.
        thetas (ndarray): Array of angles corresponding to the quadrature measurements.
        x_edges (ndarray): Edges of the bins for the quadrature measurements.
        tol (flaot, optional): tolerance. Defaults to 1e-9.
    Returns:
        counts (ndarray): 2D array where each row corresponds to a theta and each column corresponds to a bin defined by x_edges. The values are the counts of samples in each bin.
    Raises:
        ValueError: If any angle in `thetas` is not close to the corresponding angle in `thetas` within the specified tolerance.
    """
    
    thetas = np.asarray(thetas)
    n_thetas = len(thetas)
    n_bins = len(x_edges) - 1
    counts = np.zeros((n_thetas, n_bins), dtype=int)
    
    for X, th in zip(samples, thetas):
        # Find the bin index for each sample
        idx = np.argmin(np.abs(thetas - th))
        if np.abs(th - thetas[idx]) > tol:
            raise ValueError(f"Angle {th} not close to any theta in thetas. Found {thetas[idx]} instead.")
        
        # Digitize the samples into bins
        bin_indices = np.digitize(X, x_edges) - 1
        b = np.clip(bin_indices, 0, n_bins - 1)  # Ensure indices are within bounds
        counts[idx, b] += 1  # Increment the count for the corresponding bin
    return counts

counts = bin_samples(measurements, thetas, edges)
counts.shape, (counts.sum(axis=1) > 0).sum(), (counts.sum(axis=0) > 0).sum()


((1212, 60), np.int64(6), np.int64(4))

In [ ]:
def update_density_matrix(rho, R):
    """Update the density matrix using the MLE formula."""
    updated_rho = R * rho * R
    updated_rho = updated_rho / updated_rho.tr()  # Normalize the updated density matrix
    return updated_rho

def prob(rho, povm):
    """Construct the P from the density matrix rho and POVMs."""
    return np.real((rho * povm).tr())

In [ ]:
def MLE_binned(X, thetas, N=50, num_bins=60, max_iters=10, tol=1e-9):
    bins, x_edges, spectra = build_bins(N, thetas, n_bins=num_bins)
    n_thetas = len(thetas)
    

In [19]:
# def MLE_binned(counts, povms, n_iter=100, eps=1e-12, verbose=True):
#     """Maximum Likelihood Estimation for binned measurements.

#     Args:
#         counts (ndarray): Binned counts of measurements.
#         povms (list): List of POVM operators for each bin.
#         n_iter (int, optional): Number of iterations for the MLE algorithm. Defaults to 100.
#         eps (float, optional): Small value to avoid division by zero. Defaults to 1e-12.
#         verbose (bool, optional): If True, print progress. Defaults to False.
#     """
#     n_thetas, n_bins = counts.shape
#     N = povms[0][0].shape[0]  # Dimension of the Hilbert space
#     rho = qt.qeye(N) / N  # Initial guess: uniform density matrix
#     tot = counts.sum()  # Total number of counts across all bins
#     like_history = [] # Store likelihood history for convergence analysis
#     if verbose:
#         print(f"Starting MLE with {n_iter} iterations and {tot} total counts.")
#     # Iterate to update the density matrix

#     for _ in trange(n_iter, desc="MLE Iterations"):
#         # P = np.zeros_like(counts, dtype=float)  # Initialize the probability matrix
#         # for t in tqdm(range(n_thetas), desc="Theta Loop", leave=False):
#         #     # Calculate the probabilities for each bin using the POVMs
#         #     for b in tqdm(range(n_bins), desc="Bin Loop", leave=False):
#         #         P[t, b] = np.real((rho * povms[t][b]).tr())  # Calculate the probability for each bin
#         # P = np.maximum(P, eps)  # Avoid division by zero
        
#         # Update the density matrix using the counts and probabilities
#         R = qt.Qobj(np.zeros((N,N), dtype=np.complex128), dims=[[N], [N]])
#         for t in trange(n_thetas, desc="Theta Loop", leave=False):
#             for b in trange(n_bins, desc="Bin Loop", leave=False):
#                 if counts[t, b] > 0:
#                     P = np.clip(a=prob(rho, povms[t][b]), a_min=eps, a_max=None)
#                     R += counts[t, b] * povms[t][b] / P
#         R = R / tot  # Normalize the operator
#         rho = update_density_matrix(rho, R)
#         # ll = np.sum(counts * np.log(P))
#         # like_history.append(ll)
#     return rho, np.array(like_history)

# rho, hist = MLE_binned(counts=counts, povms=povms, n_iter=10, eps=1e-12, verbose=True)